In [4]:
from matplotlib.pyplot import xlabel
import torch
import h5py
import numpy as np
import networkx as nx
import scipy.sparse as sp
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.utils import to_undirected
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from ast import Num
from turtle import pd
from matplotlib.pyplot import xlabel
import torch
import h5py
import pdb
import numpy as np
import networkx as nx
import scipy.sparse as sp
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv,GATConv,GATv2Conv,SAGEConv
from torch_geometric.data import Data
from torch_geometric.utils import degree
from torch_geometric.utils import negative_sampling,get_laplacian
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
torch.set_default_tensor_type(torch.DoubleTensor)

class SAGE(nn.Module):
    def __init__(self, graph, label):
      super(SAGE, self).__init__()
      self.graph = graph
      self.label = label  #One_hot label
      self.num_nodes = graph.num_nodes
      self.edges = graph["edge_index"]
      self.in_c = 128
      self.hid_c = 64
      self.out_c = np.size(label,1)
      self.loss_function = nn.BCELoss()

      self.linear = nn.Linear(self.num_nodes, self.in_c)
      self.gcn1 = SAGEConv(self.in_c, self.hid_c)
      self.gcn2 = SAGEConv(self.hid_c, self.out_c)

    def forward(self):
      ini_feature ,edge_index = self.graph["x"],self.graph["edge_index"]
      # ini_feature = ini_feature.to(torch.float32)
      x = torch.relu(self.linear(ini_feature))
      x = torch.relu(self.gcn1(x, edge_index))
      x = self.gcn2(x, edge_index)

      return F.log_softmax(x, dim=1)

class GCN(nn.Module):
    def __init__(self, graph, label):
        super(GCN, self).__init__()
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = graph["edge_index"]
        self.in_c = 128*2
        self.hid_c = 64
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c, self.out_c)

    def forward(self):
        ini_feature ,edge_index = self.graph["x"],self.graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x = torch.relu(self.gcn1(x, edge_index))
        x = self.gcn2(x, edge_index)

        return F.log_softmax(x, dim=1)

class GAT(nn.Module):
    def __init__(self, graph, label):
        super(GAT, self).__init__()
        self.graph = graph
        self.label = label
        self.num_nodes = graph.num_nodes
        self.edges = graph["edge_index"]
        self.in_c = 128*2
        self.hid_c = 64
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GATConv(self.in_c, self.hid_c)
        self.gcn2 = GATConv(self.hid_c, self.out_c)

    def forward(self):
        ini_feature ,edge_index = self.graph["x"],self.graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x = torch.relu(self.gcn1(x, edge_index))
        x = self.gcn2(x, edge_index)

        return F.log_softmax(x, dim=1)

class MLP(nn.Module):
    def __init__(self, graph, label):
        super(MLP, self).__init__()
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = graph["edge_index"]
        self.in_c = 128*2
        self.hid_c = 64
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = nn.Linear(self.in_c, self.hid_c)
        self.gcn2 = nn.Linear(self.hid_c, self.out_c)

    def forward(self):
        ini_feature = self.graph["x"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x = torch.relu(self.gcn1(x))
        x = self.gcn2(x)

        return F.log_softmax(x, dim=1)

class MGCN(nn.Module):
    def __init__(self, graph, label):
        super(MGCN, self).__init__()
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128*2
        self.hid_c = 64
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c, self.out_c)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x = torch.relu(self.gcn1(x, edge_index,edge_weight=motif_W))
        x = self.gcn2(x, edge_index,edge_weight=motif_W)

        return F.log_softmax(x, dim=1)

class DGCN(nn.Module):
    def __init__(self, sim_graph,graph, label, cat_method = 'mlp',dropout=0):
        super(DGCN, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 64
        self.hid_c2 = 32
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()
        self.cat_method = cat_method
        self.dropout = nn.Dropout(p=dropout)

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.hid_c2)
        self.gcn3 = GCNConv(self.hid_c2, self.hid_c2*2)
        self.gcn4 = GCNConv(self.hid_c2*2, self.hid_c2)
        self.mlp = nn.Linear(self.hid_c2*2, self.out_c)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.sim_graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x = self.dropout(x)
        x1 = torch.relu(self.gcn1(x, edge_index_sim))
        x1 = self.dropout(x1)
        x2 = torch.relu(self.gcn2(x1, edge_index_sim))
        x2 = self.dropout(x2)
        x3 = torch.relu(self.gcn3(x2, edge_index,edge_weight=motif_W))
        x3 = self.dropout(x3)
        x4 = torch.relu(self.gcn4(x3, edge_index,edge_weight=motif_W))
        x4 = self.dropout(x4)
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return F.log_softmax(x_out_nc, dim=1)

class DGCN1(nn.Module):
    def __init__(self, sim_graph,graph, label):
        super(DGCN1, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 64
        self.hid_c2 = 32
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.out_c)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.sim_graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x1 = torch.relu(self.gcn1(x, edge_index,edge_weight=motif_W))
        x2 = torch.relu(self.gcn2(x1, edge_index,edge_weight=motif_W))
        return F.log_softmax(x2, dim=1)

class DGCN3(nn.Module):
    def __init__(self, sim_graph,graph, label):
        super(DGCN3, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 64
        self.hid_c2 = 32
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss() 

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.hid_c2)
        self.gcn3 = GCNConv(self.hid_c2, self.hid_c2*2)
        self.gcn4 = GCNConv(self.hid_c2*2, self.hid_c2)
        self.mlp = nn.Linear(self.hid_c2*2, self.out_c)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.sim_graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x1 = torch.relu(self.gcn1(x, edge_index,edge_weight=motif_W))
        x2 = torch.relu(self.gcn2(x1, edge_index,edge_weight=motif_W))
        x3 = torch.relu(self.gcn3(x2, edge_index,edge_weight=motif_W))
        x4 = torch.relu(self.gcn4(x3, edge_index,edge_weight=motif_W))
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return F.log_softmax(x_out_nc, dim=1)

class DGCN2(nn.Module):
    def __init__(self, sim_graph,graph, label):
        super(DGCN2, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 64
        self.hid_c2 = 32
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.hid_c2)
        self.gcn3 = GCNConv(self.hid_c2, self.hid_c2*2)
        self.gcn4 = GCNConv(self.hid_c2*2, self.hid_c2)
        self.mlp = nn.Linear(self.hid_c2*2, self.out_c)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.sim_graph["x"],self.sim_graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x1 = torch.relu(self.gcn1(x, edge_index))
        x2 = torch.relu(self.gcn2(x1, edge_index))
        x3 = torch.relu(self.gcn3(x2, edge_index))
        x4 = torch.relu(self.gcn4(x3, edge_index))
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return F.log_softmax(x_out_nc, dim=1)

class DGCN4(nn.Module):
    def __init__(self, sim_graph, graph, label):
        super(DGCN4, self).__init__()
        self.sim_graph = sim_graph
        self.graph = graph
        self.label = label  #One_hot label
        self.num_nodes = graph.num_nodes
        self.edges = sim_graph["edge_index"]
        self.node_degree = degree(self.graph["edge_index"][0]).t()
        self.in_c = 128
        self.hid_c = 64
        self.hid_c2 = 32
        self.out_c = np.size(label,1)
        self.loss_function = nn.BCELoss()

        self.linear = nn.Linear(self.num_nodes, self.in_c)
        self.gcn1 = GCNConv(self.in_c, self.hid_c)
        self.gcn2 = GCNConv(self.hid_c,self.hid_c2)
        self.gcn3 = GCNConv(self.in_c, self.hid_c)
        self.gcn4 = GCNConv(self.hid_c, self.hid_c2)
        self.mlp = nn.Linear(self.hid_c2*2, self.out_c)

    def forward(self):
        ini_feature ,edge_index,motif_W = self.sim_graph["x"],self.graph["edge_index"],self.graph["edge_attr"]
        edge_index_sim= self.sim_graph["edge_index"]
        # ini_feature = ini_feature.to(torch.float32)
        x = torch.relu(self.linear(ini_feature))
        x1 = torch.relu(self.gcn1(x, edge_index_sim))
        x2 = torch.relu(self.gcn2(x1, edge_index_sim))
        x3 = torch.relu(self.gcn3(x, edge_index,edge_weight=motif_W))
        x4 = torch.relu(self.gcn4(x3, edge_index,edge_weight=motif_W))
        x_out = torch.cat((x2,x4),1)
        x_out_nc = self.mlp(x_out)
        return F.log_softmax(x_out_nc, dim=1)

In [6]:
def Get_edge_pyg(A):

  edge_index_temp = sp.coo_matrix(A)
  values = edge_index_temp.data
  indices = np.vstack((edge_index_temp.row, edge_index_temp.col))
  # edge_index_A = torch.LongTensor(indices)
  edge = torch.LongTensor(indices)
  edge_attribute = torch.FloatTensor(values)
  edge_index = torch.sparse_coo_tensor(edge, edge_attribute, edge_index_temp.shape)
  return edge_index, edge, edge_attribute

def graph_info(graph):
  print("Graph Information:\nnum_nodes:"+str(graph.num_nodes)+"  num_edges:"+str(int(graph.num_edges/2))+"  is_directed:"+str(graph.is_directed())+"\nnode_att_num:"+str(graph.num_node_features)+"  edge_att_num:"+str(graph.num_edge_features)+"  isolated_nodes:"+str(graph.has_isolated_nodes()))
  return

def One_hot_label(label):
  One_hot = np.zeros((len(label), np.max(label)-np.min(label)+1 ))
  for i in range(len(label)):
      One_hot[i][label[i]]=1
  return One_hot

def load_mat(feature,name):
    mat = feature[str(name)][:]
    mat = np.transpose(mat)
    mat = torch.tensor(mat)
    return mat

def Compute_accruracy(out,Label,test_set):
  pred = out[test_set].argmax(axis=1)
  true = Label[test_set].argmax(axis=1)
  correct = int((pred.numpy()==true).sum())
  acc = correct / len(test_set)
  # print('Accuracy: {:.4f}'.format(acc))
  return acc

In [7]:
torch.set_default_tensor_type(torch.DoubleTensor)
path = './NC_M3.mat'
feature = h5py.File(path)


# cora_A = load_mat(feature,'cora_A')
#citeseer
# cora_A = load_mat(feature,'cora_A')
# cora_M32 = load_mat(feature,'cora_M32')
# cora_A = load_mat(feature,'citeseer_A')
# cora_M32 = load_mat(feature,'citeseer_M32')
# cora_A = load_mat(feature,'pumed')
# cora_M32 = load_mat(feature,'pumed_M32')
# cora_A = load_mat(feature,'flickr_A')
# cora_M32 = load_mat(feature,'flickr_M32')


cora_A = load_mat(feature,'Blog_A')
cora_M32 = load_mat(feature,'Blog_M32')

_,edge,_ = Get_edge_pyg(cora_A)
_,edge_M32_AW,M32_att_AW = Get_edge_pyg(cora_A+cora_M32)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_size = 0.8
epochs = 100
learning_rate = 0.001
seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
path1 = './B_Labels.npy'
labels = np.load(path1)
Label = One_hot_label(labels)
Nodes = list(range(len(labels)))

graph_cora = Data(x=torch.tensor(cora_A), edge_index = edge).to(device)
graph_cora_M32_AW = Data(x=(cora_A+cora_M32), edge_index = edge_M32_AW, edge_attr = M32_att_AW).to(device)

train_set, test_set = train_test_split(Nodes, test_size=test_size, random_state=41)
train_label = torch.tensor(labels[train_set])
true_label = torch.tensor(labels[test_set])

In [ ]:
print(graph_cora.num_nodes)
print(true_label.shape)
print(train_label.shape)
print(Label.shape)

In [ ]:
model = MGCN(graph=graph_cora, label = Label).to(device)
print(f"test_size:{test_size}")

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
model.train()
for epoch in range(epochs):
  optimizer.zero_grad()
  out = model.forward()
  pred_label = out[train_set].to(device)
  loss = F.nll_loss(input=pred_label.to(device),target=train_label.to(device))
  loss.backward()
  optimizer.step()
model.eval()
S = model.forward()
acc=Compute_accruracy(S.cpu(),Label,test_set)
print(f"{model}--acc:{acc:.4f}")